<a href="https://colab.research.google.com/github/lcipolina/Ray_tutorials/blob/main/RLLIB_MARL_Empty_action_Ray_2_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This example creates a simple multi-agent custom environment and trains it using Tune and Air.

Its aim is to investigate the generation of empty action_dictionaries coming from RLLIB's policy.

In [ ]:
!pip install ray[rllib]==2.2.0 --quiet

In [ ]:
''' FOR NEWER RLLIB ONLY
!pip install gymnasium --quiet
import gymnasium as gym
'''

' FOR NEWER RLLIB ONLY\n!pip install gymnasium --quiet\nimport gymnasium as gym\n'

In [ ]:
import os
import numpy as np
import ray
from ray.rllib.env import MultiAgentEnv
from gym.spaces import MultiDiscrete
from ray import air, tune
from gym.spaces import Tuple, Box, MultiDiscrete, Discrete
from ray.rllib.algorithms.ppo import PPOConfig

/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


In [ ]:
# Simple MARL custom Environment

'''Simple Multi-agent environment to test things
'''


class TurnEnv(MultiAgentEnv):

    def __init__(self, config=None):
        super().__init__()
        self.num_agents              = 2
        self.t                       = 0
        self.agent_lst               = list(range(self.num_agents))
        self._agent_ids              = set(self.agent_lst)
        self.dones_dict              = {agent: False for agent in self.agent_lst}
        self.dones_dict['__all__']   = False
        self.observation_space       = Discrete(self.num_agents)
        self.action_space            = self.observation_space

    def reset(self):
        return {0: self.observation_space.sample()}  # {agent:obs}


    def step(self, action_dict):
        
        self.t +=1
        
     
        if not action_dict:
           print("EMPTY ACTION DICT!!!")
           print('self.t =', self.t)
       
        if self.t == 10: 
           self.dones_dict              = {agent: True for agent in self.agent_lst}
           self.dones_dict['__all__']   = True

        return {1:self.observation_space.sample()},\
               {1:1},\
               self.dones_dict, \
               {} #{agent:obs}, {agent:rews}, dones, info




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Configure experiments and train

In [ ]:
def train_ray():

    #Register custom env
    env_name = 'TurnEnv'
    tune.register_env(env_name, lambda env_ctx: TurnEnv()) #the register_env needs a callable/iterable

    #Experiment configuration
    NUM_CPUS = os.cpu_count()

    config = PPOConfig()\
    .framework("torch")\
    .rollouts(num_rollout_workers=1, observation_filter="MeanStdFilter")\
    .resources(num_gpus=0,num_cpus_per_worker=1)\
    .evaluation(evaluation_interval=2,evaluation_duration = 2, evaluation_duration_unit='episodes',
                evaluation_config= {"explore": False})\
    .environment(env = TurnEnv, env_config={
                                     "num_workers": NUM_CPUS - 1,
                                     "disable_env_checking":True} 

                )


   


    train_steps = 1
    experiment_name = 'my_experiment'
    tuner = tune.Tuner("PPO", param_space=config,
                              run_config=air.RunConfig(
                                        name =  experiment_name,
                                        stop={"timesteps_total": train_steps},
                                        checkpoint_config=air.CheckpointConfig(checkpoint_frequency=1, checkpoint_at_end=True),
                                        verbose= 0
                                )
                     )
    results = tuner.fit()

    #check_learning_achieved(results, stop_reward)
    df = results.get_dataframe()
    best_result = results.get_best_result(metric="episode_reward_mean", mode="max")
    #print(best_result)


    

  

In [ ]:
if ray.is_initialized(): ray.shutdown()
ray.init(local_mode=True,include_dashboard=False, ignore_reinit_error=True) 

train_ray()

ray.shutdown()

print('done')

2023-05-01 17:09:29,460	WARNING worker.py:837 -- `ray.get_gpu_ids()` will always return the empty list when called from the driver. This is because Ray does not manage GPU allocations to the driver process.
:task_name:bundle_reservation_check_func
:actor_name:PPO
2023-05-01 17:09:29,617	WARNING algorithm_config.py:488 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-05-01 17:09:29,631	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


:task_name:bundle_reservation_check_func
:actor_name:PPO


:actor_name:RolloutWorker
2023-05-01 17:09:29,792	WARNING multi_agent_env.py:266 -- observation_space_sample() of <TurnEnv instance> has not been implemented. You can either implement it yourself or bring the observation space into the preferred format of a mapping from agent ids to their individual observation spaces. 
2023-05-01 17:09:29,794	WARNING multi_agent_env.py:160 -- observation_space_contains() of <TurnEnv instance> has not been implemented. You can either implement it yourself or bring the observation space into the preferred format of a mapping from agent ids to their individual observation spaces. 
2023-05-01 17:09:29,800	WARNING multi_agent_env.py:160 -- observation_space_contains() of <TurnEnv instance> has not been implemented. You can either implement it yourself or bring the observation space into the preferred format of a mapping from agent ids to their individual observation spaces. 
2023-05-01 17:09:29,807	WARNING multi_agent_env.py:160 -- observation_space_contai

:actor_name:RolloutWorker
EMPTY ACTION DICT!!!
self.t = 1
EMPTY ACTION DICT!!!
self.t = 1
:actor_name:PPO


:actor_name:PPO
/usr/local/lib/python3.10/dist-packages/ray/rllib/utils/filter.py:84: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  self.mean_array = np.zeros(shape)


done
